# [Link to dataset](https://data.mendeley.com/datasets/zw4p9kj6nt/2)

In [1]:
%load_ext autotime

!nvidia-smi -L

import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-3b49e2b8-87f0-c515-798b-3492ec05a183)
GPU 1: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-07628ed7-6ef8-fd67-7d03-cb6a89f72de4)


In [2]:
import numpy as np, tensorflow as tf, matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools, glob

# Experiment tracking with mlflow
import mlflow
import mlflow.tensorflow as mltf

```
mkdir mlflow && mlflow server \
    --backend-store-uri sqlite:///mlflow/mlflow.db \
    --default-artifact-root ./mlflow/artifacts \
    --host 0.0.0.0
```

In [3]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mosquito")
mltf.autolog()

2022/08/16 03:26:48 INFO mlflow.tracking.fluent: Experiment with name 'mosquito' does not exist. Creating a new experiment.


In [4]:
train_path = "./dataset/data_splitting/Train/"
valid_path = "./dataset/data_splitting/Test/"
test_path = "./dataset/data_splitting/Pred/"

# How to save ImageDataGenerator parameters in mlflow ?

In [5]:
# You can add more augmentations, if you want

train_gen = ImageDataGenerator(
    rotation_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
)

gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [6]:
targetMap='''aegypti landing
aegypti smashed
albopictus landing
albopictus smashed
Culex landing
Culex smashed'''.split('\n')
targetMap

['aegypti landing',
 'aegypti smashed',
 'albopictus landing',
 'albopictus smashed',
 'Culex landing',
 'Culex smashed']

In [7]:
# Hyper-Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 1
NUM_CLASSES = len(targetMap)

In [8]:
train = train_gen.flow_from_directory(train_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
valid = gen.flow_from_directory(valid_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
test = gen.flow_from_directory(test_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)

Found 4200 images belonging to 6 classes.
Found 1799 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


In [9]:
# Pretrained Model

ptm = VGG16(input_shape=list(IMG_SIZE)+[3], weights='imagenet', include_top=False)
ptm.trainable = False


vgg = Model(ptm.input, ptm.output)
vgg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
i = layers.Input(shape=(IMG_SIZE[:]+(3,)))
ptm = vgg(i)

# p = layers.Rescaling(1./255)(i)
# ptm = vgg(p)

x = layers.Flatten()(ptm)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(i, x)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080

# How to save Compile parameters in mlflow ?

In [11]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

h = model.fit(
    train,
    validation_data=valid,
    epochs=EPOCHS
)

2022/08/16 03:26:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e22e6c5370374437bb3fc8f3a008b2f4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


132/132 [==============================] - 40s 276ms/step - loss: 2.4872 - acc: 0.4190 - val_loss: 0.9443 - val_acc: 0.6643


INFO:tensorflow:Assets written to: /tmp/tmp5rf1a5km/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp5rf1a5km/model/data/model/assets
2022/08/16 03:27:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp5rf1a5km/model, flavor: keras), fall back to return ['tensorflow==2.9.0', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.
